In [ ]:
#install packages

!pip uninstall torchtext
!pip install torchtext==0.6.0

Found existing installation: torchtext 0.6.0
Uninstalling torchtext-0.6.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchtext-0.6.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchtext/*
Proceed (Y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#upload the file in the drive
from google.colab import files
uploaded = files.upload()

Saving bbc-text.csv to bbc-text (4).csv


In [ ]:

import pandas as pd
import io
df = pd.read_csv(io.StringIO(uploaded['bbc-text.csv'].decode('utf-8')))
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:

!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#downloading  the glove embedding
! kaggle datasets download -d takuok/glove840b300dtxt

glove840b300dtxt.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#unzipping the zip file

!unzip glove840b300dtxt.zip

Archive:  glove840b300dtxt.zip
replace glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


we define the TEXT and LABEL fields using torchtext.data.Field.
These fields define the processing steps for the text and label data.

We load the dataset
The dataset is split into train, validation, and test sets using the split method.

The vocabulary is built using TEXT.build_vocab with a no maximum size and pre-trained word vectors (GloVe).

We create data iterators using data.BucketIterator.splits to create batches of data during training, validation, and testing.



In [ ]:
#Preparation on the dataset into torchtext object


import torch
from torchtext import data
from torchtext.vocab import Vectors
import spacy
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

batch_size=32
max_sen_len = None

class Dataset(object):
    def __init__(self, ):
        self.train_iterator = None
        self.test_iterator = None
        self.val_iterator = None
        self.vocab = []
        self.word_embeddings = {}

    def get_pandas_df(self, filename):
        '''
        Load the data into Pandas.DataFrame object
        This will be used to convert data to torchtext object
        '''
        full_df = pd.read_csv(io.StringIO(uploaded['bbc-text.csv'].decode('utf-8')))
        full_df.rename(columns = {'category':'label'}, inplace = True)
        c = full_df.label.astype('category')
        d = dict(enumerate(c.cat.categories))
        full_df['label'] = full_df.label.astype('category').cat.codes
        return full_df

    def load_data(self, w2v_file, train_file, test_file=None, val_file=None):
        '''
        Loads the data from files
        Sets up iterators for training, validation and test data
        Also create vocabulary and word embeddings based on the data

        Inputs:
            w2v_file (String): path to file containing word embeddings (GloVe/Word2Vec)
            train_file (String): path to training file
            test_file (String): path to test file
            val_file (String): path to validation file
        '''

        NLP = spacy.load('en_core_web_sm')
        tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]

        # Creating Field for data
        TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, fix_length=max_sen_len)
        LABEL = data.Field(sequential=False, use_vocab=False)
        datafields = [("label",LABEL),("text",TEXT)]

        # Load data from pd.DataFrame into torchtext.data.Dataset
        train_df = self.get_pandas_df(train_file)
        train_examples = [data.Example.fromlist(i, datafields) for i in train_df.values.tolist()]
        train_data = data.Dataset(train_examples, datafields)

        #get test dataset
        train_data, test_data = train_data.split(split_ratio=0.8)
        #get Validation dataset
        val_data, test_data = test_data.split(split_ratio=0.8)

        if w2v_file:
            TEXT.build_vocab(train_data, vectors=Vectors(w2v_file))
        self.word_embeddings = TEXT.vocab.vectors
        self.vocab = TEXT.vocab

        self.train_iterator = data.BucketIterator(
            (train_data),
            batch_size=batch_size,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=True)

        self.val_iterator, self.test_iterator = data.BucketIterator.splits(
            (val_data, test_data),
            batch_size=batch_size,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=False)

        print ("Loaded {} training examples".format(len(train_data)))
        print ("Loaded {} test examples".format(len(test_data)))
        print ("Loaded {} validation examples".format(len(val_data)))

In [ ]:
import sys
import torch.optim as optim
from torch import nn
import torch
import io

w2v_file = 'glove.840B.300d.txt'

#creating the Dataset Object
dataset = Dataset()
#loading the data and Sets up iterators
dataset.load_data(w2v_file, 'bbc-text.csv')


Loaded 1780 training examples
Loaded 89 test examples
Loaded 356 validation examples


In [ ]:
next(iter(dataset.train_iterator))


[torchtext.data.batch.Batch of size 32]
	[.label]:[torch.LongTensor of size 32]
	[.text]:[torch.LongTensor of size 812x32]

Seq2Seq model is defined in the RNN class.
It consists of an embedding layer, an LSTM layer, and a linear layer for classification.
Dropout is applied for regularization.

The model is instantiated with the specified dimensions and moved to the appropriate device (GPU if available).

The loss function (cross-entropy) and optimizer (Adam) are defined.

The train function is defined to perform a single training epoch, which has model.train() and iterates over the training data, calculates predictions, loss, and accuracy, performs backpropagation, and updates the model parameters.

The evaluate function is defined to evaluate the model on the validation or test data using model.eval() and iterates over the data to calculate loss and accuracy without backpropagation.

The categorical_accuracy function calculates the accuracy of the model's predictions.

In [ ]:

# Define the RNN model architecture
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = self.dropout(embedded)
        output, (hidden, cell) = self.rnn(embedded)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(hidden)
        predictions = self.fc(hidden.squeeze(0))
        return predictions

# Instantiate the model
input_dim = len(dataset.vocab)
embedding_dim = 300
hidden_dim = 256
output_dim = 5
n_layers = 2
dropout = 0.5
model = RNN(input_dim, embedding_dim, hidden_dim, output_dim, n_layers, dropout)

# Copy pre-trained word embeddings to the model's embedding layer
pretrained_embeddings = dataset.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
def train(model, iterator, optimizer, criterion):
    model.train()

    epoch_loss = 0
    epoch_acc = 0

    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Evaluate the model
def evaluate(model, iterator, criterion):
    model.eval()

    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Define the accuracy metric
def categorical_accuracy(preds, y):
    _, predicted = torch.max(preds, 1)
    correct = (predicted == y).float()
    acc = correct.sum() / len(correct)
    return acc



In [ ]:
# Train the model

#During each epoch, the training loss and accuracy are calculated using the train function, and the validation loss and accuracy are calculated using the evaluate function.
#If the validation loss is lower than the best seen so far, the model's state is saved to a file.

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq_model.pt')

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')


In [ ]:

# Load the best model and test
model.load_state_dict(torch.load('seq2seq_model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
